In [1]:
from sympy import symbols, diff, sin
import math
import numpy as np
from scipy import integrate

#VARIAVEIS DO PROBLEMA
elementos = [
    [1, 2, 6, 5],
    [2, 3, 7, 6],
    [3, 4, 8, 7]
]
#NUMERO DE ELEMENTOS
nel = 3
#NUMERO DE NOS
N_no = 8
#NOS PRESCRITOS
N_p = [1,4,5,8]
#NUMERO DE EQUACOES
N_eq = N_no - len(N_p)
#CONSTANTE
k = 1

# COORDENADA X DE CADA ELEMENTO
x_no_el_1 = [0, 1/3, 1/3, 0]
x_no_el_2 = [1/3, 2/3, 2/3, 1/3]
x_no_el_3 = [2/3, 1, 1, 2/3]

# COORDENADA Y DE CADA ELEMENTO
y_no_el_1 = [0, 0, 1/3, 1/3]
y_no_el_2 = [0, 0, 1/3, 1/3]
y_no_el_3 = [0, 0, 1/3, 1/3]

# MATRIZ JACOBIANA
def calcular_jacobiano(xi, eta, x_no_el, y_no_el):
    x_xi = (1/4) * np.dot([-(1-eta), (1-eta), (1+eta), -(1+eta)], x_no_el)
    y_xi = (1/4) * np.dot([-(1-eta), (1-eta), (1+eta), -(1+eta)], y_no_el)
    x_eta = (1/4) * np.dot([-(1-xi), -(1+xi), (1+xi), (1-xi)], x_no_el)
    y_eta = (1/4) * np.dot([-(1-xi), -(1+xi), (1+xi), (1-xi)], y_no_el)
    
    J = np.array([
        [x_xi, x_eta],
        [y_xi, y_eta]
    ])
    return J

#FUNCAO U(X) DO PROBLEMA
def u(x): 
    return 100*np.sin((np.pi*x)/2)

#FUNCOES BASE
def phi(a, xi, eta):
    if a == 1: 
        return 0.25*(1-xi)*(1-eta)
    if a == 2: 
        return 0.25*(1+xi)*(1-eta)
    if a == 3: 
        return 0.25*(1+xi)*(1+eta)
    if a == 4: 
        return 0.25*(1-xi)*(1+eta)

#GRADIENTE DAS FUNCOES BASE
def gradiente_phi(a, xi, eta):
    if a == 1: 
        return np.array([-0.25*(1-eta), -0.25*(1-xi)])
    if a == 2: 
        return np.array([0.25*(1-eta), -0.25*(1+xi)])
    if a == 3: 
        return np.array([0.25*(1+eta), 0.25*(1+xi)])
    if a == 4: 
        return np.array([-0.25*(1+eta), 0.25*(1-xi)])

#PESOS E PONTOS DE GAUSS
pontos_gauss = [-1/np.sqrt(3), 1/np.sqrt(3)]
pesos_gauss = [1.0, 1.0]

#MATRIZ K2 E K2 OBTIDAS DA DESCRICAO DA ALTERNATIVA C
K2 = np.array([
    [2/3, -1/6, -1/3, -1/6],
    [-1/6, 2/3, -1/6, -1/3],
    [-1/3, -1/6, 2/3, -1/6],
    [-1/6, -1/3, -1/6, 2/3]
])

K3 = K2

In [13]:
#CONSTRUINDO VETOR EQ
def definir_vetorEQ(N_no, N_p):
    vetorEQ = np.zeros(N_no, dtype=int)
    contador = 1
    for i in range(N_no):
        if (i + 1) not in N_p:
            vetorEQ[i] = contador
            contador += 1
        else:
            vetorEQ[i] = 0
    return vetorEQ


vetorEQ = definir_vetorEQ(N_no, N_p)
print("VETOR EQ:\n")
print(vetorEQ)

VETOR EQ:

[0 1 2 0 0 3 4 0]


In [14]:
#CONSTRUINDO MATRIZ LG
def definir_matrizLG(elementos):
    matrizLG = np.array(elementos)
    return matrizLG.T

matrizLG = definir_matrizLG(elementos)

print("MATRIZ LG:\n")
print(matrizLG)

MATRIZ LG:

[[1 2 3]
 [2 3 4]
 [6 7 8]
 [5 6 7]]


In [4]:
# MATRIZ LOCAL K1
def calcular_matriz_local_K1():
    K = np.zeros((4,4))
    for a in range(4):
        for b in range(4):
            soma = 0
            for i in range(2):
                for j in range(2):
                    #APLICANDO PONTOS E PESOS DE GAUSS
                    xi = pontos_gauss[i]
                    eta = pontos_gauss[j]
                    wi = pesos_gauss[i]
                    wj = pesos_gauss[j]

                    # CALCULO DA MATRIZ JABIANA, INVERSA E DETERMINATE
                    J = calcular_jacobiano(xi, eta, x_no_el_1, y_no_el_1)
                    J_inv = np.linalg.inv(J)
                    J_inv_T = J_inv.T
                    det_J = np.linalg.det(J)

                    #CALCULO DO GRADIENTE
                    gradiente_phi_a = gradiente_phi(a+1, xi, eta)
                    gradiente_phi_b = gradiente_phi(b+1, xi, eta)

                    #PASSANDO PARA O DOMINIO OMEGA B BARRA
                    gradiente_phi_a_omega_b = np.dot(J_inv_T, gradiente_phi_a)
                    gradiente_phi_b_omega_b = np.dot(J_inv_T, gradiente_phi_b)

                    soma += wi*wj*k*np.dot(gradiente_phi_a_omega_b, gradiente_phi_b_omega_b)*det_J
            K[a, b] = soma
    return K


K1 = calcular_matriz_local_K1()
print("MATRIZ LOCAL K1:")
print(np.round(K1, 4))

Matriz local K1:
[[ 0.6667 -0.1667 -0.3333 -0.1667]
 [-0.1667  0.6667 -0.1667 -0.3333]
 [-0.3333 -0.1667  0.6667 -0.1667]
 [-0.1667 -0.3333 -0.1667  0.6667]]


In [19]:
# CALCULO DOS VETORES FORÇA LOCAIS
def calcular_soma_de_integrais_f_e(x_no_el, y_no_el):
    S = np.zeros((4, 4))
    for a in range(4):
        for b in range(4):
            soma = 0
            for i in range(2):
                for j in range(2):
                    
                    #APLICANDO PONTOS E PESOS DE GAUSS
                    xi = pontos_gauss[i]
                    eta = pontos_gauss[j]
                    wi = pesos_gauss[i]
                    wj = pesos_gauss[j]
                    
                    # CALCULO DA MATRIZ JABIANA
                    J = calcular_jacobiano(xi, eta, x_no_el, y_no_el)
                    det_J = np.linalg.det(J)
                    
                    soma += wi*wj*phi(a+1, xi, eta)*phi(b+1, xi, eta)*det_J
            S[a, b] = soma
    return S

#FAZENDO A SOMA DA QUADRATURA GAUSSIANA PARA CADA ELEMENTO PARA DEPOIS MULTIPLICAR POR F APLICADA EM CADA NO/PONTO DO ELEMENTO
soma_de_integrais_el1 = calcular_soma_de_integrais_f_e(x_no_el_1, y_no_el_1)
soma_de_integrais_el2 = calcular_soma_de_integrais_f_e(x_no_el_2, y_no_el_2)
soma_de_integrais_el3 = calcular_soma_de_integrais_f_e(x_no_el_3, y_no_el_3)

#FUNCAO QUE APLICA A FUNCAO F EM CADA NO/PONTO DO ELEMENTO
def f_em_cada_no(x_no):
    return 25 * np.pi**2 * np.sin(np.pi * np.array(x_no) / 2)

#MULTIPLICANDO CADA SOMA PELA F APLICADA EM CADA NO/PONTO DO ELEMENTO
f1 = soma_de_integrais_el1.dot(f_em_cada_no(x_no_el_1))
f2 = soma_de_integrais_el2.dot(f_em_cada_no(x_no_el_2))
f3 = soma_de_integrais_el3.dot(f_em_cada_no(x_no_el_3))

#PARTE PRESCRITA
p1 = np.zeros(4)
p2 = np.zeros(4)
p3 = np.array([0, 100, 100, 0])

#RESULTADO FINAL DE CADA VETOR FORCA LOCAL
F1 =(f1 - K1.dot(p1)).reshape(4,1)
F2 = (f2 - K2.dot(p2)).reshape(4,1)
F3 = (f3 - K3.dot(p3)).reshape(4,1)


print("VETOR F1:\n")
print(np.round(F1, 4))
print("\nVETOR F2:\n")
print(np.round(F2, 4))
print("\nVETOR F3:\n")
print(np.round(F3, 4))

VETOR F1:

[[1.1423]
 [2.2846]
 [2.2846]
 [1.1423]]

VETOR F2:

[[4.2632]
 [5.0994]
 [5.0994]
 [4.2632]]

VETOR F3:

[[ 56.2417]
 [-43.4522]
 [-43.4522]
 [ 56.2417]]


In [20]:
#CALCULO DA MATRIZ GLOBAL K
matrizes_locais_Ke = [K1, K2, K3]

def montar_matriz_global_K(EQ,LG, matrizes_locais_Kes): 
    K = np.zeros((N_eq, N_eq))
    for e in range(nel):  
        for a in range(4):  
            for b in range(4):
                i = int(EQ[LG[a, e] - 1])  
                j = int(EQ[LG[b, e] - 1]) 
                if i != 0 and j != 0: 
                    K[i - 1, j - 1] += matrizes_locais_Ke[e][a, b]
    return K.round(4)


matriz_global_K =  montar_matriz_global_K(vetorEQ, matrizLG, matrizes_locais_Ke)
print("MATRIZ K GLOBAL:\n")
print(matriz_global_K)

MATRIZ K GLOBAL:

[[ 1.3333 -0.1667 -0.3333 -0.3333]
 [-0.1667  1.3333 -0.3333 -0.3333]
 [-0.3333 -0.3333  1.3333 -0.1667]
 [-0.3333 -0.3333 -0.1667  1.3333]]


In [22]:
#CALCULO DO VETOR FORÇA GLOBAL
vetores_locais_Fe = [F1, F2, F3]

def montar_vetor_forca_global_F(EQ,LG,vetores_locais_Fe):
    F = np.zeros((N_eq, 1))
    for e in range(nel):
        Fe = vetores_locais_Fe[e].reshape((4, 1))
        for a in range(4):  
            i = EQ[LG[a, e] - 1] 
            if i != 0:
                F[i - 1, 0] += Fe[a, 0]
    return F

vetor_global_F = montar_vetor_forca_global_F(vetorEQ, matrizLG, vetores_locais_Fe)
print("VETOR FORÇA GLOBAL F:\n")
print(np.round(vetor_global_F, 4))

VETOR FORÇA GLOBAL F:

[[ 6.5478]
 [61.3411]
 [ 6.5478]
 [61.3411]]


In [23]:
#SOLUCAO
c = np.linalg.solve(matriz_global_K, vetor_global_F).round(4)
print("\nVETOR SOLUÇÃO APROXIMADA:\n" )
print(c)


VETOR SOLUÇÃO APROXIMADA:

[[49.6245]
 [86.1534]
 [49.6245]
 [86.1534]]
